# Notebook 05: Model Training and Evaluation mit bereinigtem Datensatz

In diesem Notebook werden der bereinigte Datensatz ohne `mmr` und redundante Features geladen, in Trainings- und Testset aufgeteilt und anschliessend zwei Baseline-Modelle (Lineare Regression und Random Forest) trainiert, mittels Kreuzvalidierung evaluiert und abschliessend auf dem Testset verglichen.

## 1. Setup & Laden der Daten
In diesem Kapitel werden alle benoetigten Bibliotheken importiert und der bereinigte Datensatz eingelesen sowie ein erster Ueberblick gegeben.

In [2]:
# 1.1 Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib

# 1.2 Konstanten
RANDOM_STATE = 42
DATA_PATH_CLEAN = "../data/processed/cars_features_no_mmr_reduced.csv"

# 1.3 Einlesen des bereinigten Datensatzes
df_cleaned = pd.read_csv(DATA_PATH_CLEAN)

# 1.4 Erster Ueberblick
print(f"Shape des bereinigten DataFrames: {df_cleaned.shape}\n")
display(df_cleaned.head())


Shape des bereinigten DataFrames: (91562, 22)



,year,condition,odometer,sale_year,sale_month,sale_day,sale_weekday,body,transmission,color,...,season,has_sport,has_limited,has_lx,has_se,has_touring,has_premium,miles_per_year,color_popularity,sellingprice
0,2015,2.0,5559.0,2015,1,13,1,Sedan,automatic,white,...,Winter,0,0,0,1,0,0,5559.0,4,12000.0
1,2012,35.0,45035.0,2014,12,18,3,SUV,automatic,gray,...,Winter,0,1,0,0,0,0,22517.5,3,14100.0
2,2012,46.0,20035.0,2014,12,18,3,SUV,automatic,gray,...,Winter,0,0,0,1,0,0,10017.5,3,20800.0
3,2012,46.0,41115.0,2014,12,18,3,SUV,automatic,white,...,Winter,0,0,0,1,0,0,20557.5,4,22100.0
4,2012,3.0,26747.0,2014,12,17,2,Hatchback,automatic,red,...,Winter,0,0,0,0,0,0,13373.5,6,14000.0


## 2. Train-/Test-Split
In diesem Kapitel werden die Merkmale und die Zielvariable definiert und der Datensatz in Trainings- und Testset im Verhältnis 80/20 aufgeteilt.

In [3]:
# 2.1 Merkmale (X) und Ziel (y) definieren
X = df_cleaned.drop("sellingprice", axis=1)
y = df_cleaned["sellingprice"]

# 2.2 Aufteilen in Trainings- und Testset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# Kontrolle der Formen
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (73249, 21), X_test: (18313, 21)
y_train: (73249,), y_test: (18313,)


## 3. Preprocessing-Pipeline
Dieser Abschnitt definiert und baut die Vorverarbeitungs-Pipeline für numerische und kategoriale Features mithilfe von Imputer, Skalierer und One-Hot-Encoding auf.

In [4]:
# 3.1 Numerische und kategoriale Features identifizieren
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=["object"]).columns.tolist()

print("Numerische Features:", numeric_features)
print("Kategoriale Features:", categorical_features)

# 3.2 Pipelines definieren
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# 3.3 ColumnTransformer erstellen
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_features),
    ("cat", categorical_pipeline, categorical_features)
])


Numerische Features: ['year', 'condition', 'odometer', 'sale_year', 'sale_month', 'sale_day', 'sale_weekday', 'avg_price_state', 'has_sport', 'has_limited', 'has_lx', 'has_se', 'has_touring', 'has_premium', 'miles_per_year', 'color_popularity']
Kategoriale Features: ['body', 'transmission', 'color', 'interior', 'season']


## 4. Baseline-Modelle & Cross-Validation
In diesem Abschnitt werden zwei Baseline-Modelle (Lineare Regression und Random Forest) mit 5-fach Kreuzvalidierung auf den Trainingsdaten evaluiert. Als Metrik wird der negative Root Mean Squared Error (neg_root_mean_squared_error) verwendet.

### 4.1 Lineare Regression: Pipeline und 5-fach CV

In [5]:
pipe_lr = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

scores_lr = cross_val_score(
    pipe_lr,
    X_train,
    y_train,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
print(f"CV RMSE (Linear Regression): {(-scores_lr).mean():.2f} (+/- {scores_lr.std():.2f})")


CV RMSE (Linear Regression): 2746.44 (+/- 15.45)


### 4.2 Random Forest: Pipeline und 5-fach CV

In [6]:
pipe_rf = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

scores_rf = cross_val_score(
    pipe_rf,
    X_train,
    y_train,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
print(f"CV RMSE (Random Forest): {(-scores_rf).mean():.2f} (+/- {scores_rf.std():.2f})")


CV RMSE (Random Forest): 2452.95 (+/- 22.81)


## 4. Ergebnisse der Kreuzvalidierung

| Modell                | CV RMSE        | Std. Dev.    |
|-----------------------|----------------|--------------|
| Lineare Regression    | 2 746.44       | ± 15.45      |
| Random Forest         | 2 452.95       | ± 22.81      |

**Interpretation:**  
- Der Random Forest erzielt mit einem durchschnittlichen RMSE von **2 452.95** eine deutlich bessere Vorhersagegüte als die Lineare Regression (RMSE **2 746.44**).  
- Die höhere Streuung der RF-Ergebnisse (± 22.81 vs. ± 15.45) deutet auf eine etwas variablere Performance zwischen den Folds hin, bleibt jedoch im Rahmen.  
- Insgesamt spricht das niedrigere RMSE des Random Forest für dessen Verwendung als baselines Modell im weiteren Verlauf.  

## 5. Modelltraining & Evaluation
In diesem Kapitel werden beide Modelle auf dem vollständigen Trainingsset trainiert, Vorhersagen auf Trainings- und Testset erstellt und die Metriken MAE, RMSE und R² berechnet.

### 5.0: Vorbereitung calc_metrics

In [7]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def calc_metrics(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2   = r2_score(y_true, y_pred)
    return mae, rmse, r2


### 5.1 Linear Regression: Training & Evaluation

In [8]:
# Pipeline erstellen und trainieren
pipe_lr = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])
pipe_lr.fit(X_train, y_train)

# Vorhersagen
y_pred_train_lr = pipe_lr.predict(X_train)
y_pred_test_lr  = pipe_lr.predict(X_test)

# Metriken berechnen
mae_tr_lr, rmse_tr_lr, r2_tr_lr = calc_metrics(y_train, y_pred_train_lr)
mae_te_lr,  rmse_te_lr,  r2_te_lr  = calc_metrics(y_test,  y_pred_test_lr)

# Ergebnisse ausgeben
print("Linear Regression:")
print(f"  Train -> MAE: {mae_tr_lr:.2f}, RMSE: {rmse_tr_lr:.2f}, R²: {r2_tr_lr:.4f}")
print(f"  Test  -> MAE: {mae_te_lr:.2f}, RMSE: {rmse_te_lr:.2f}, R²: {r2_te_lr:.4f}")


Linear Regression:
  Train -> MAE: 2122.81, RMSE: 2743.33, R²: 0.6922
  Test  -> MAE: 2115.34, RMSE: 2725.81, R²: 0.6967


### 5.2 Random Forest: Training & Evaluation

In [9]:
# Pipeline erstellen und trainieren
pipe_rf = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=100,
        random_state=RANDOM_STATE,
        n_jobs=1
    ))
])
pipe_rf.fit(X_train, y_train)

# Vorhersagen
y_pred_train_rf = pipe_rf.predict(X_train)
y_pred_test_rf  = pipe_rf.predict(X_test)

# Metriken berechnen
mae_tr_rf, rmse_tr_rf, r2_tr_rf = calc_metrics(y_train, y_pred_train_rf)
mae_te_rf,  rmse_te_rf,  r2_te_rf  = calc_metrics(y_test,  y_pred_test_rf)

# Ergebnisse ausgeben
print("Random Forest:")
print(f"  Train -> MAE: {mae_tr_rf:.2f}, RMSE: {rmse_tr_rf:.2f}, R²: {r2_tr_rf:.4f}")
print(f"  Test  -> MAE: {mae_te_rf:.2f}, RMSE: {rmse_te_rf:.2f}, R²: {r2_te_rf:.4f}")


Random Forest:
  Train -> MAE: 671.21, RMSE: 912.90, R²: 0.9659
  Test  -> MAE: 1804.50, RMSE: 2441.23, R²: 0.7567


## 5. Ergebnisse des Modelltrainings & Evaluation

| Modell               | Set    | MAE      | RMSE     | R²      |
|----------------------|--------|----------|----------|---------|
| **Lineare Regression** | Train  | 2 122.81 | 2 743.33 | 0.6922  |
|                      | Test   | 2 115.34 | 2 725.81 | 0.6967  |
| **Random Forest**     | Train  |   671.21 |   912.90 | 0.9659  |
|                      | Test   | 1 804.50 | 2 441.23 | 0.7567  |

**Interpretation:**  
- **Lineare Regression**  
  - Die Performance auf Training und Test ist nahezu identisch (ΔRMSE ≈ 17.5), R²-Werte um 0.69 zeigen moderate Modellgüte ohne Over- oder Underfitting.  
  - Der Fehler liegt bei knapp 2 700 USD RMSE – dieses Modell erklärt rund 69 % der Varianz der Verkaufspreise.

- **Random Forest**  
  - Sehr niedrige Fehler auf den Trainingsdaten (RMSE ≈ 913, R² ≈ 0.97) im Vergleich zum Testset (RMSE ≈ 2 441, R² ≈ 0.76) deutet auf Overfitting hin.  
  - Dennoch übertrifft der Random Forest die lineare Regression deutlich auf dem Testset (RMSE um ~280 USD geringer, R² um ~0.06 höher).

**Schlussfolgerung:**  
- Die lineare Regression liefert eine stabile, aber weniger genaue Vorhersage.  
- Der Random Forest erreicht bessere Genauigkeit auf neuen Daten, zeigt jedoch Overfitting-Tendenzen.  
- Für produktive Nutzung sollte ggf. das Random Forest-Modell weiter abgestimmt (z. B. Hyperparameter-Tuning, Regularisierung) oder alternative Modelle (z. B. Gradient Boosting) geprüft werden.


## 6. Feature Importance je Modell
Ermittlung und Darstellung der wichtigsten Merkmale für die Lineare Regression (über Koeffizienten) und für den Random Forest (über `feature_importances_`).

In [10]:
# 6.1 Feature-Namen aus dem Preprocessor extrahieren
feature_names = preprocessor.get_feature_names_out()

# Sicherstellen, dass die Pipeline trainiert ist
pipe_lr.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)

# 6.2 Lineare Regression: Koeffizienten
coef = pipe_lr.named_steps["model"].coef_
import pandas as pd

fi_lr = pd.DataFrame({
    "feature": feature_names,
    "importance": np.abs(coef)
}).sort_values("importance", ascending=False)

print("Top 10 Features (Lineare Regression):")
print(fi_lr.head(10).to_string(index=False))


Top 10 Features (Lineare Regression):
                feature  importance
cat__interior_off-white 3706.695862
    cat__body_Hatchback 3123.800793
        cat__color_pink 2822.760402
          cat__body_suv 2629.268936
      cat__body_Minivan 2125.148139
          cat__body_SUV 2099.717886
   cat__color_off-white 2094.526383
        cat__body_Sedan 2085.124716
              num__year 2021.848934
   cat__interior_yellow 1921.087853


In [11]:
# 6.3 Random Forest: Feature Importances
importances = pipe_rf.named_steps["model"].feature_importances_

fi_rf = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

print("\nTop 10 Features (Random Forest):")
print(fi_rf.head(10).to_string(index=False))



Top 10 Features (Random Forest):
              feature  importance
        num__odometer    0.406584
            num__year    0.145629
        cat__body_SUV    0.128833
       num__condition    0.060692
  num__miles_per_year    0.039499
    cat__body_Minivan    0.029617
 num__avg_price_state    0.029565
        num__sale_day    0.026950
num__color_popularity    0.013841
    num__sale_weekday    0.012602


## 6. Zusammenfassung der Feature Importances und Massnahmen gegen Overfitting

### Wichtige Erkenntnisse
- **Lineare Regression**  
  - Die Top-Koeffizienten entstammen primär kategorialen Variablen, etwa `interior_off-white`, `body_Hatchback` und `color_pink`, gefolgt von `year`.  
  - Numerische Merkmale wie `odometer` tauchen in den Top-10 der LR-Koeffizienten nicht auf, obwohl sie im RF wichtig sind – hier scheint die LR die Einflüsse anders zu gewichten.

- **Random Forest**  
  - Dominant ist das Merkmal `odometer` (ca. 41 % des Gesamt-Importance-Gewichts), gefolgt von `year` (15 %) und `body_SUV` (13 %).  
  - `condition`, `miles_per_year` und `avg_price_state` bringen ebenfalls einen messbaren Beitrag.  

### Potenzielle Massnahmen gegen Overfitting
1. **Feature-Reduktion**  
   - **Remove Rare Categories**: Kategorien mit sehr kleinem Vorkommen (z. B. `interior_yellow`, `color_pink`) zusammenfassen oder in eine "Other"-Gruppe überführen, um Rauschen zu verringern.  
   - **Low-Importance Features**: Features mit fast null Importance im RF (z. B. `sale_weekday`, farbliche Kategorien jenseits der Top-10) ausschliessen und erneut validieren.

2. **Regularisierung / Modellkomplexität**  
   - **Max Depth / Min Samples**: Für den RF tieferes `max_depth` und höheres `min_samples_leaf` wählen, um den Einfluss des dominanten `odometer`-Merkmals abzumildern.  
   - **Max Features**: `max_features="sqrt"` oder kleiner einstellen, damit nicht in jedem Split fast ausschliesslich `odometer` und `year` ausgewählt werden.

3. **Feature-Engineering**  
   - **Interaktionsmerkmale**: Prüfen, ob gezielte Interaktionen (z. B. `odometer * condition`) sinnvoll sind und Overfitting nicht verschärfen.  
   - **Binning**: Starker Einfluss von `odometer` könnte in Diskretisierung (z. B. Kilometer-Bins) gegossen werden, um Ausreisserrobustheit zu erhöhen.

4. **Cross-Validation & Ensembling**  
   - **Stärkere CV**: 10-fach-CV oder wiederholte CV (RepeatedKFold), um stabilere Schätzungen zu erhalten.  
   - **Ensemble aus unterschiedlichen Modellen**: Lineare Regression und RF kombinieren (z. B. durch Stacking), um die Ausprägungen einzelner Modelle auszubalancieren.

5. **Alternative Modelle**  
   - **Gradient Boosting** (z. B. LightGBM mit Regularisierung über `lambda_l1`, `lambda_l2`): Oft robuster gegen ein dominantes Merkmal wie `odometer`.  
   - **Lasso / ElasticNet**: Für die lineare Variante, um unwichtige Koeffizienten direkt auf null zu setzen.

---

Durch gezieltes Entfernen oder Zusammenfassen weniger informativer Kategorien sowie Anpassung der RF-Hyperparameter kann die Modellkomplexität reduziert und Overfitting nachhaltig bekämpft werden. Anschliessend empfiehlt sich eine erneute Validierung (z. B. Repeated CV), um den Effekt der Änderungen zu prüfen.  
